# CTE: Common Table Expression

In [2]:
import mysql.connector
import pandas as pd

In [8]:
mydb = mysql.connector.connect(
host = 'localhost',
user = 'root',
passwd = '*',
database = 'sakila')

In [9]:
kursor = mydb.cursor()
def sql(query) :
    kursor.execute(query)
    hasil = kursor.fetchall()
    data = pd.DataFrame(data = hasil, columns = kursor.column_names)
    return data

In [10]:
sql('''
select * from actor;
''')

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33
...,...,...,...,...
195,196,BELA,WALKEN,2006-02-15 04:34:33
196,197,REESE,WEST,2006-02-15 04:34:33
197,198,MARY,KEITEL,2006-02-15 04:34:33
198,199,JULIA,FAWCETT,2006-02-15 04:34:33


In [11]:
sql('select * from (select first_name, last_name from actor) as nama;') # table expression

,first_name,last_name
0,PENELOPE,GUINESS
1,NICK,WAHLBERG
2,ED,CHASE
3,JENNIFER,DAVIS
4,JOHNNY,LOLLOBRIGIDA
...,...,...
195,BELA,WALKEN
196,REESE,WEST
197,MARY,KEITEL
198,JULIA,FAWCETT


CTE adalah subquery terpisah yang disimpan sebagai tabel sementara dan bisa digunakan berulangkali. 

Untuk membuat CTE, gunakan keyword WITH <nama tabel> as ( ...)

Contoh: menampilkan film_id dan retal durasinya, di mana durasi film yang ditampilkan adalah yang di atas rata-rata durasi film semua film

In [13]:
# subquery

sql('''
select film_id, rental_duration from film
where length > (select avg (length) from film)
order by rental_duration desc;
''')

,film_id,rental_duration
0,27,7
1,36,7
2,70,7
3,80,7
4,88,7
...,...,...
484,946,3
485,973,3
486,984,3
487,987,3


In [14]:
# CTE

sql('''
WITH rata_lama AS (
SELECT AVG(length) as avg_lama from film
)

select film_id, rental_duration from film
where length> (select avg_lama from rata_lama)
order by 2 desc;
''') # order by 2, 2 merujuk pada posisi kolom yang dipanggil (rental_duration)

,film_id,rental_duration
0,27,7
1,36,7
2,70,7
3,80,7
4,88,7
...,...,...
484,946,3
485,973,3
486,984,3
487,987,3


In [18]:
# CTE bisa dipakai berulang dalam 1 query

sql('''
with rata_panjang as(
select rating, avg(length) as avg_length
from film
group by 1
)
select title, length from film 
where length between (
select avg_length from rata_panjang where rating='g') 
and (select avg_length from rata_panjang where rating='pg-13');
''')
# film yang lamanya di antara rata-rata lama film dgn rating 'g' dan rata-rata lama film dengan rating 'pg-13'

,title,length
0,AFFAIR PREJUDICE,117
1,ALABAMA DEVIL,114
2,AMADEUS HOLY,113
3,APOCALYPSE FLAMINGOS,119
4,ATTACKS HATE,113
...,...,...
65,TROOPERS METAL,115
66,UNTOUCHABLES SUNRISE,120
67,WAIT CIDER,112
68,WATERSHIP FRONTIER,112


In [19]:
sql('''
select rating, avg(length) as avg_length
from film
group by 1
''') # isi dari CTE rata_panjang

,rating,avg_length
0,PG,112.0052
1,G,111.0506
2,NC-17,113.2286
3,PG-13,120.4439
4,R,118.6615


In [16]:
sql('select * from film limit 5;')

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Behind the Scenes, Deleted Scenes}",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"{Trailers, Deleted Scenes}",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"{Commentaries, Behind the Scenes}",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,{Deleted Scenes},2006-02-15 05:03:42


In [20]:
# membuat lebih dari 1 CTE dalam 1 query

sql('''
WITH rata_durasi AS(
select avg(rental_duration) as avg_durasi from film
),
rata_lama AS(
select avg(length) as avg_length from film
)

select title, length, rental_duration from film
where rental_duration > (select avg_durasi from rata_durasi)
and length<(select avg_length from rata_lama);
''')

# film-film dengan rental duration di atas rata-rata dan length di bawah rata-rata

,title,length,rental_duration
0,ACADEMY DINOSAUR,86,6
1,ADAPTATION HOLES,50,7
2,AIRPLANE SIERRA,62,6
3,AIRPORT POLLOCK,54,6
4,ALADDIN CALENDAR,63,6
...,...,...,...
288,WONKA SEA,85,6
289,WYOMING STORM,100,6
290,YENTL IDAHO,86,5
291,ZHIVAGO CORE,105,6


In [21]:
mydb = mysql.connector.connect(
host = 'localhost',
user = 'root',
passwd = '*',
database = 'world')

In [22]:
kursor = mydb.cursor()

# Windows Function

In [24]:
sql( 'select * from country;')

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,JosÃ© Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,ShqipÃ«ria,Republic,Rexhep Mejdani,34.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,YEM,Yemen,Asia,Middle East,527968.0,1918.0,18112000,59.8,6041.0,5729.0,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.0,1918.0,10640000,72.4,17000.0,NaN,Jugoslavija,Federal Republic,Vojislav KoÂštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910.0,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964.0,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162.0,ZM


In [27]:
sql('select name, population, avg(population) from country')

# rata-rata populasi dunia, tanpa over()

,name,population,avg(population)
0,Aruba,103000,25434098.1172


In [26]:
# dengan over
# rata- rata populasi jadi kolom baru
sql('''
select name, population , avg(population) over() as rata_populasi
from country;
''')

,name,population,rata_populasi
0,Aruba,103000,25434098.1172
1,Afghanistan,22720000,25434098.1172
2,Angola,12878000,25434098.1172
3,Anguilla,8000,25434098.1172
4,Albania,3401200,25434098.1172
...,...,...,...
234,Yemen,18112000,25434098.1172
235,Yugoslavia,10640000,25434098.1172
236,South Africa,40377000,25434098.1172
237,Zambia,9169000,25434098.1172


In [29]:
# contoh, menghitung selisih antara populasi suatu negara dengan rata-rata populasi dunia

sql('''
select name, population,
avg(population) over() as rata_populasi,
population-avg(population) over() as selisih_pop
from country
order by 4;
''')

,name,population,rata_populasi,selisih_pop
0,Antarctica,0,25434098.1172,-25434098.1172
1,French Southern territories,0,25434098.1172,-25434098.1172
2,Bouvet Island,0,25434098.1172,-25434098.1172
3,Heard Island and McDonald Islands,0,25434098.1172,-25434098.1172
4,British Indian Ocean Territory,0,25434098.1172,-25434098.1172
...,...,...,...,...
234,Brazil,170115000,25434098.1172,144680901.8828
235,Indonesia,212107000,25434098.1172,186672901.8828
236,United States,278357000,25434098.1172,252922901.8828
237,India,1013662000,25434098.1172,988227901.8828


In [35]:
# group by

sql('''
select continent, avg(population) from country
group by 1;
''')

,continent,avg(population)
0,North America,13053864.8649
1,Asia,72647562.7451
2,Africa,13525431.0345
3,Europe,15871186.9565
4,South America,24698571.4286
5,Oceania,1085755.3571
6,Antarctica,0.0000


In [36]:
# over partition
# perhatikan isi kolom rata_pop
sql('''
select name, continent, 
avg(population) over (partition by continent) as rata_pop
from country;
''')

# membuat kolom baru yang berisi rata-rata populasi berdasarkan benua.

,name,continent,rata_pop
0,Afghanistan,Asia,72647562.7451
1,United Arab Emirates,Asia,72647562.7451
2,Armenia,Asia,72647562.7451
3,Azerbaijan,Asia,72647562.7451
4,Bangladesh,Asia,72647562.7451
...,...,...,...
234,Peru,South America,24698571.4286
235,Paraguay,South America,24698571.4286
236,Suriname,South America,24698571.4286
237,Uruguay,South America,24698571.4286


In [37]:
sql('''
select name, continent, population,
avg(population) over(partition by continent) as rata_pop,
population-avg(population) over(partition by continent) as selisih_pop
from country;
''')
# membuat kolom baru berisi selisih antara populasi negara dengan rata-rata populasi benua tempat negara itu berada

,name,continent,population,rata_pop,selisih_pop
0,Afghanistan,Asia,22720000,72647562.7451,-49927562.7451
1,United Arab Emirates,Asia,2441000,72647562.7451,-70206562.7451
2,Armenia,Asia,3520000,72647562.7451,-69127562.7451
3,Azerbaijan,Asia,7734000,72647562.7451,-64913562.7451
4,Bangladesh,Asia,129155000,72647562.7451,56507437.2549
...,...,...,...,...,...
234,Peru,South America,25662000,24698571.4286,963428.5714
235,Paraguay,South America,5496000,24698571.4286,-19202571.4286
236,Suriname,South America,417000,24698571.4286,-24281571.4286
237,Uruguay,South America,3337000,24698571.4286,-21361571.4286


## ROW_NUMBER()

In [39]:
# membuat kolom baru berisikan nomor urut baris

sql('''
select name, continent, row_number() over() as nomor
from country;
''')

,name,continent,nomor
0,Aruba,North America,1
1,Afghanistan,Asia,2
2,Angola,Africa,3
3,Anguilla,North America,4
4,Albania,Europe,5
...,...,...,...
234,Yemen,Asia,235
235,Yugoslavia,Europe,236
236,South Africa,Africa,237
237,Zambia,Africa,238


In [41]:
pd.set_option('display.max_rows', None) # biar barisnya kelihatan semua

sql('''
select name, continent, 
row_number() over(partition by continent)
from country;
''')

,name,continent,row_number() over(partition by continent)
0,Afghanistan,Asia,1
1,United Arab Emirates,Asia,2
2,Armenia,Asia,3
3,Azerbaijan,Asia,4
4,Bangladesh,Asia,5
5,Bahrain,Asia,6
6,Brunei,Asia,7
7,Bhutan,Asia,8
8,China,Asia,9
9,Cyprus,Asia,10


In [42]:
# elaborasi dengan CTE, untuk mengambil hanya 3 negara pertama per benua

sql('''
WITH nomor_baris as(
    select name, continent,
    ROW_NUMBER() OVER(PARTITION BY continent) as nomor
    from country
)

select * from nomor_baris
where nomor<=3
''')

,name,continent,nomor
0,Afghanistan,Asia,1
1,United Arab Emirates,Asia,2
2,Armenia,Asia,3
3,Albania,Europe,1
4,Andorra,Europe,2
5,Austria,Europe,3
6,Aruba,North America,1
7,Anguilla,North America,2
8,Netherlands Antilles,North America,3
9,Angola,Africa,1


## RANK() and DENSE_RANK()
membuat ranking, berdasarkan nilai tertentu

In [46]:
# membuat baris baru berisi ranking negara berdasarkan populasinya

sql('''
select name, continent, population, 
RANK() OVER(ORDER BY population) as rank_pop
from country;
''')

,name,continent,population,rank_pop
0,Antarctica,Antarctica,0,1
1,French Southern territories,Antarctica,0,1
2,Bouvet Island,Antarctica,0,1
3,Heard Island and McDonald Islands,Antarctica,0,1
4,British Indian Ocean Territory,Africa,0,1
5,South Georgia and the South Sandwich Islands,Antarctica,0,1
6,United States Minor Outlying Islands,Oceania,0,1
7,Pitcairn,Oceania,50,8
8,Cocos (Keeling) Islands,Oceania,600,9
9,Holy See (Vatican City State),Europe,1000,10


In [45]:
sql('''
select name, continent, population, 
DENSE_RANK() OVER(ORDER BY population) as rank_pop
from country;
''')
#beda rank dan dense rank adalah dari cara pemberian ranking, pada rank, kalau ada ranking yang sama, 
# ranking berikutnya akan diskip sesuai jumlah ranking yang sama, pada dense rank, angka ranking dilanjutkan

,name,continent,population,rank_pop
0,Antarctica,Antarctica,0,1
1,French Southern territories,Antarctica,0,1
2,Bouvet Island,Antarctica,0,1
3,Heard Island and McDonald Islands,Antarctica,0,1
4,British Indian Ocean Territory,Africa,0,1
5,South Georgia and the South Sandwich Islands,Antarctica,0,1
6,United States Minor Outlying Islands,Oceania,0,1
7,Pitcairn,Oceania,50,2
8,Cocos (Keeling) Islands,Oceania,600,3
9,Holy See (Vatican City State),Europe,1000,4


In [47]:
# rank dengan partition by

sql('''
select name, continent, population,
RANK() OVER(PARTITION by continent ORDER BY POPULATION)
from country;
''')

,name,continent,population,RANK() OVER(PARTITION by continent ORDER BY POPULATION)
0,Maldives,Asia,286000,1
1,Brunei,Asia,328000,2
2,Macao,Asia,473000,3
3,Qatar,Asia,599000,4
4,Bahrain,Asia,617000,5
5,Cyprus,Asia,754700,6
6,East Timor,Asia,885000,7
7,Kuwait,Asia,1972000,8
8,Bhutan,Asia,2124000,9
9,United Arab Emirates,Asia,2441000,10


In [49]:
sql('''
select continent, count(*) as jumlah_negara, avg(population) as rata_pop,
rank() over(order by avg(population) desc) as rank_pop,
rank() over(order by count(*) desc) as rank_total
from country
group by continent
order by rank_pop;
''')

# rank dengan group by

,continent,jumlah_negara,rata_pop,rank_pop,rank_total
0,Asia,51,72647562.7451,1,2
1,South America,14,24698571.4286,2,6
2,Europe,46,15871186.9565,3,3
3,Africa,58,13525431.0345,4,1
4,North America,37,13053864.8649,5,4
5,Oceania,28,1085755.3571,6,5
6,Antarctica,5,0.0000,7,7


## NTILE 
Membuat kolom baru berisikan nomor sesuai yang kita tentukan. Pembagian nomor didasarkan pada urutan nilai data.  
Misalkan ada 20 data, kita buat 4 ntile, maka 5 data dengan nilai terkecil (default kecil-besar) akan diberi angka 1, 5 data berikutnya, diberi angka 2, dan 5 data dengan nilai terbesar diberi angka 4.
 

In [51]:
sql('''
with quantile as(
select name, population, 
ntile(4) over(order by population desc) as quantile_pop
from country
)
select * from quantile where quantile_pop=1;
''')

# 25% negara dengan jumlah populasi terbesar

,name,population,quantile_pop
0,China,1277558000,1
1,India,1013662000,1
2,United States,278357000,1
3,Indonesia,212107000,1
4,Brazil,170115000,1
5,Pakistan,156483000,1
6,Russian Federation,146934000,1
7,Bangladesh,129155000,1
8,Japan,126714000,1
9,Nigeria,111506000,1


## SLIDING WINDOWS
untuk mencari nilai aggregate dari beberapa data berurutan, biasanya dipakai untuk mencari moving average atau cumulative sum

In [52]:
sql('''
with total_negara as(
    select continent, count(*) as total_negara from country
    group by 1
)
select continent, total_negara, 
SUM(total_negara) OVER (ORDER BY total_negara DESC
ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW) as total_cumulatif
from total_negara;

''')

# UNBOUNDED PRECEDING --> baris pertama
# CURRENT ROW --> baris aktif

,continent,total_negara,total_cumulatif
0,Africa,58,58
1,Asia,51,109
2,Europe,46,155
3,North America,37,192
4,Oceania,28,220
5,South America,14,234
6,Antarctica,5,239


In [58]:
sql('''
with total_negara_benua as(
    select continent, count(*) as total_negara
    from country
    group by 1
)
select continent, total_negara,
sum(total_negara) over (order by total_negara desc
ROWS BETWEEN CURRENT ROW and 1 FOLLOWING) as moving_cumulatif
from total_negara_benua;

''')

# penggunaan preceding dan following, perlu dicantumkan berapa baris setelah/sebelum
# di contoh ini kita menjumlahkan total negara tiap baris dengan baris berikutnya

,continent,total_negara,moving_cumulatif
0,Africa,58,109
1,Asia,51,97
2,Europe,46,83
3,North America,37,65
4,Oceania,28,42
5,South America,14,19
6,Antarctica,5,5
